# Wikipedia Notable Life Expectancies

# [Notebook 5 of : Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean4_thanak_2022_06_23.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean3.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean3", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 132652 rows and 23 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,economist,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
132650,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2,2022,June,1980,None,volleyball player,Olympic champion coach,None,None,None,None,None,None,None,None,69.0,None,Russia,None
132651,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,engineer,member of the Chinese Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
42024,4,Bronisław Żurakowski,", 98, Polish aerospace engineer.",https://en.wikipedia.org/wiki/Bronis%C5%82aw_%C5%BBurakowski,0,2009,October,None,None,aerospace engineer,None,None,None,None,None,None,None,None,None,98.0,None,Poland,None
89285,2,Jaroslav Kořán,", 77, Czech translator, writer and politician, Charter 77 signatory, Mayor of Prague .",https://en.wikipedia.org/wiki/Jaroslav_Ko%C5%99%C3%A1n,3,2017,June,1990 1991,None,translator,writer politician,Charter 77 signatory,Mayor of Prague,None,None,None,None,None,None,77.0,None,Czech Republic,None
76359,5,Sir Philip Goodhart,", 89, British politician, MP for Beckenham .",https://en.wikipedia.org/wiki/Philip_Goodhart,2,2015,July,1957 1992,None,politician,MP for Beckenham,None,None,None,None,None,None,None,None,89.0,None,United Kingdom of Great Britain and Northern Ireland,None
86708,23,Serhiy Tovstoplet,", 79, Ukrainian Olympic swimmer .",https://en.wikipedia.org/wiki/Serhiy_Tovstoplet,2,2017,January,1960,None,Olympic swimmer,None,None,None,None,None,None,None,None,None,79.0,None,Ukraine,None
131321,5,Paul Siebel,", 84, American singer-songwriter.",https://en.wikipedia.org/wiki/Paul_Siebel,7,2022,April,None,None,singer songwriter,None,None,None,None,None,None,None,None,None,84.0,None,United States of America,None


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132652 entries, 0 to 132651
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   day             132652 non-null  object 
 1   name            132652 non-null  object 
 2   info            132652 non-null  object 
 3   link            132652 non-null  object 
 4   num_references  132652 non-null  object 
 5   year            132652 non-null  int64  
 6   month           132652 non-null  object 
 7   info_parenth    49830 non-null   object 
 8   info_1          35 non-null      object 
 9   info_2          132604 non-null  object 
 10  info_3          62571 non-null   object 
 11  info_4          12605 non-null   object 
 12  info_5          1497 non-null    object 
 13  info_6          216 non-null     object 
 14  info_7          31 non-null      object 
 15  info_8          6 non-null       object 
 16  info_9          1 non-null       object 
 17  info_10   

<IPython.core.display.Javascript object>

#### Loading `nation_map` from Pickle File to Dictionary nation_map

In [6]:
# Load the nation_map
with open("nation_map.pkl", "rb") as f:
    nation_map = pickle.load(f)

<IPython.core.display.Javascript object>

#### Loading `other_species` list from other_species.csv

In [7]:
# Loading other_species list
other_species_df = pd.read_csv("other_species.csv")
other_species = other_species_df["species"].tolist()
other_species = list(
    set(other_species)
)  # To avoid searching duplicates as we have been adding back to same csv file

<IPython.core.display.Javascript object>

#### Observations:
- With our dataframe, `nation_map`, and `other_species` list loaded, we can proceed to extracting the other features.
- First, we will clean up the divided `info` columns by removing any remaining digits and nationality and country values.
- We will use the same functions from previous notebooks.

#### Function to Save Indices of Rows Matching Regular Expressions Pattern to a List and Print Number of Rows with Match 

In [8]:
# Define a function that takes dataframe, column name, and re pattern as arguments and returns list of indices
# for which column value matches re pattern
def rows_with_pattern(dataframe, column, pattern):
    """
    Takes input of dataframe, column name, and re pattern 
    and returns list of indices for rows that contain match
    for pattern anywhere within value for given column.
    
    dataframe: dataframe
    column: column name
    pattern: re pattern
    """
    index_list = []

    for i in dataframe.index:
        item = dataframe.loc[i, column]
        match = re.search(pattern, item)
        if match:
            index_list.append(i)
    print(
        f"There are {len(index_list)} rows with matching pattern in column '{column}'."
    )
    return index_list

<IPython.core.display.Javascript object>

#### Function to Use rows_with_pattern Function for Multiple Regular Expression Patterns

In [9]:
# Define a function that calls rows_with_pattern function for multiple re patterns
# returning a single list of indices for all rows with any pattern match


def multiple_patterns(dataframe, column, patterns):
    """
    Takes input dataframe, column, and list of re patterns and returns single list 
    of indices for rows in which a match for any pattern is found with re.search
    
    dataframe: dataframe
    column: column name
    patterns: list of re patterns
    """
    rows_combined = []

    # For loop to check each pattern
    for pattern in patterns:

        # List and number of rows matching each pattern
        print(pattern)
        rows_to_check = rows_with_pattern(dataframe, column, pattern)
        print("")

        # Add list for each pattern to combined list
        rows_combined += rows_to_check

    return rows_combined

<IPython.core.display.Javascript object>

### Removing Remaining Digits and Nationality/Country Values from Divided `info` Columns

#### List of Columns to Treat

In [10]:
# List of columns to treat
cols_lst = [
    "info_1",
    "info_2",
    "info_3",
    "info_4",
    "info_5",
    "info_6",
    "info_7",
    "info_8",
    "info_9",
    "info_10",
    "info_11",
    "info_parenth",
]

<IPython.core.display.Javascript object>

#### Removing Digits

In [11]:
# Regular expression for parenthesis and its contents
pattern = r"\d"

# For loop to find indices of rows that have pattern
rows_combined = []
for column in cols_lst:
    dataframe = df[df[column].notna()]
    rows_to_check = rows_with_pattern(dataframe, column, pattern)
    rows_combined += rows_to_check

# Checking a sample of rows
df.loc[rows_combined, :].sample(2)

There are 0 rows with matching pattern in column 'info_1'.
There are 442 rows with matching pattern in column 'info_2'.
There are 2252 rows with matching pattern in column 'info_3'.
There are 1060 rows with matching pattern in column 'info_4'.
There are 69 rows with matching pattern in column 'info_5'.
There are 5 rows with matching pattern in column 'info_6'.
There are 0 rows with matching pattern in column 'info_7'.
There are 0 rows with matching pattern in column 'info_8'.
There are 0 rows with matching pattern in column 'info_9'.
There are 0 rows with matching pattern in column 'info_10'.
There are 0 rows with matching pattern in column 'info_11'.
There are 24403 rows with matching pattern in column 'info_parenth'.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
127724,2,Giuseppe Chiaretti,", 88, Italian Roman Catholic prelate, archbishop of Perugia–Città della Pieve .",https://en.wikipedia.org/wiki/Giuseppe_Chiaretti,3,2021,December,1995 2009,None,Roman Catholic prelate,archbishop of Perugia Città della Pieve,None,None,None,None,None,None,None,None,88.0,None,Italy,Italy
110035,25,Paul Goma,", 84, Romanian writer, COVID-19.",https://en.wikipedia.org/wiki/Paul_Goma,23,2020,March,None,None,writer,COVID 19,None,None,None,None,None,None,None,None,84.0,None,Romania,None


<IPython.core.display.Javascript object>

In [12]:
# For loop to extract digits
for column in cols_lst:
    for index in set(rows_combined):
        item = df.loc[index, column]
        if item:
            match = re.search(pattern, item)
            if match:
                df.loc[index, column] = re.sub(pattern, "", item)

# Rechecking number and example rows after treatment
# For loop to find indices of rows that have pattern
recheck_rows = []
for column in cols_lst:
    dataframe = df[df[column].notna()]
    rows_to_check = rows_with_pattern(dataframe, column, pattern)
    recheck_rows += rows_to_check

There are 0 rows with matching pattern in column 'info_1'.
There are 0 rows with matching pattern in column 'info_2'.
There are 0 rows with matching pattern in column 'info_3'.
There are 0 rows with matching pattern in column 'info_4'.
There are 0 rows with matching pattern in column 'info_5'.
There are 0 rows with matching pattern in column 'info_6'.
There are 0 rows with matching pattern in column 'info_7'.
There are 0 rows with matching pattern in column 'info_8'.
There are 0 rows with matching pattern in column 'info_9'.
There are 0 rows with matching pattern in column 'info_10'.
There are 0 rows with matching pattern in column 'info_11'.
There are 0 rows with matching pattern in column 'info_parenth'.


<IPython.core.display.Javascript object>

#### Removing Any Remaining Matches with  `nation_map` Keys and Values

In [13]:
%%time

# For loop to extract remaining information matching items in nation_map
for column in cols_lst:
    dataframe = df[df[column].notna()]
    for nationality, country in nation_map.items():
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if nationality + ' ' in item or country + ' ' in item or item.endswith(nationality) or item.endswith(country):
                    df.loc[index, column] = item.replace(nationality, "").strip().replace(country,'').strip()

CPU times: total: 19min 54s
Wall time: 19min 55s


<IPython.core.display.Javascript object>

#### Observations:
- After that bit of tidying, we can proceed to extracting `known_for_1` values.
- The bulk of these values should be in `info_2`, according to the Wikipedia defined fields, so we will start there.

## Extracting `known_for` Data
Our goal will be to have some broader categories into which the specific values will fit.  `known_for` is a diverse feature, in that an individual may be known for a long-term role or roles, a specific event, a relationship with another person who is famous, etc.  So, to some extent we will see what we find and adapt as we go.

Also, we will abandon searching left to right as an individual may fit more than one category, and in no particular order.  For example, Ronald Reagan, is entered as "American actor and politician".  He is most known as the 40th president of The United States, so if we prioritized the first value, he would fit only into the category containing actor.  At the same time, it may have been his acting career that led to his political career.  Both arenas are relevant, so we will aim to capture all categories for an individual.  Later, when there are duplicate categories for an indivual, we can remove the redundant values.

We will take the following approach:
1. create and check a list of unique values in `info_2` that have a minimum number repeated, sufficient to create sets for each category, but not so exhaustive to be time prohibitive to manually enter.
2. using the pop() method, add each role to it's associated category's set, below.
3. combine the sets for each category into one dictionary.
4. search for the values in the dictionary and extract the category key value to a new column `known_for_1`, `known_for_2`, etc.

In [14]:
# Obtaining values for column and their counts
col_values = df["info_2"].value_counts()

# Creating a list for values that occur more than set number of time
roles_list = [index for index in col_values.index if col_values[index] > 30]

# Checking length of list
print(f"We will examine the top {len(roles_list)} unique values in info_2.")

We will examine the top 447 unique values in info_2.


<IPython.core.display.Javascript object>

In [15]:
# # Using pop to check list items and add to associated dictionary below
# roles_list.pop()

<IPython.core.display.Javascript object>

In [16]:
# Creating lists for each category
politics_govt_law = [
    "politician",
    "economist",
    "attorney",
    "trade unionist",
    "unionist",
    "aristocrat",
    "diplomat",
    "lawyer",
    "activist",
    "civil rights",
    "federal",
    "judge",
    "political",
    "politics",
    "royal",
    "civil servant",
    "jurist",
    "judge",
    "conservationist",
    "government official",
    "government",
    "barrister",
    "militant",
    "environmentalist",
    "public servant",
    "King",
    "Queen",
    "Princess",
    "Prince",
    "President",
    "Prime Minister",
    "leader",
    "Nazi",
    "Administration",
    "Ambassador",
    "ambassador",
]

arts = [
    "actor",
    "dancer",
    "choreographer",
    "model",
    "television",
    "jazz",
    "singer",
    "composer",
    "conductor",
    "journalist",
    "writer",
    "saxophonist",
    "film director",
    "comedian",
    "photojournalist",
    "poet",
    "actress",
    "film",
    "editor",
    "drummer",
    "producer",
    "songwriter",
    "publisher",
    "author",
    "violinist",
    "rapper",
    "musician",
    "animator",
    "filmmaker",
    "pianist",
    "historian",
    "comic",
    "screenwriter",
    "fashion",
    "designer",
    "guitarist",
    "voice",
    "opera",
    "cinematographer",
    "playwright",
    "sculptor",
    "novelist",
    "photographer",
    "architect",
    "painter",
    "artist",
    "disc jockey",
    "dj",
    "DJ",
    "MC",
    "bridge player",
    "tenor",
    "trombonist",
    "filmmaker",
    "ballerina",
    "bassist",
    "film critic",
    "critic",
    "personality",
    "organist",
    "operatic",
    "lyricist",
    "translator",
    "visual artist",
    "soprano",
    "cellist",
    "broadcaster",
    "chef",
    "literary critic",
    "ballet",
    "illustrator",
    "theatre director",
    "trumpeter",
    "presenter",
    "sportscaster",
    "cartoonist",
    "sportswriter",
    "choral",
    "music",
    "arts",
    "dance",
]
sports = [
    "football",
    "footballer",
    "Olympic",
    "skier",
    "hockey",
    "soccer",
    "cricket",
    "soccer",
    "sprinter",
    "equestrian",
    "gymnast",
    "fencer",
    "chess",
    "wrestler",
    "swimmer",
    "basketball",
    "hurler",
    "sailor",
    "rower",
    "rugby",
    "athlete",
    "golfer",
    "boxer",
    "tennis",
    "cyclist",
    "racing",
    "driver",
    "cricketer",
    "baseball",
    "speedway rider",
    "speedway",
    "rider",
    "badminton",
    "sport shooter",
    "runner",
    "umpire",
    "judoka",
    "volleyball",
    "track and field",
    "track",
    "bobsledder",
    "canoer",
    "bodybuilder",
    "skater",
    "curler",
    "Olympic diver",
    "martial artist",
    "racer",
    "handball",
    "ski jumper",
    "racehorse trainer",
    "racecar driver",
    "hurdler",
    "polo",
    "Olympic shooter",
    "weightlifter",
    "Baseball",
    "mountaineer",
    "jockey",
    "Olympic sports shooter",
    "referee",
    "general manager",
    "sports",
    "sport",
    "athletics",
    "athletic",
]
sciences = [
    "engineer",
    "physicist",
    "geologist",
    "psychiatrist",
    "botanist",
    "biologist",
    "anthropologist",
    "astronomer",
    "biochemist",
    "scientist",
    "computer",
    "programmer",
    "archaeologist",
    "psychologist",
    "sociologist",
    "physician",
    "chemist",
    "physicist",
    "mathematician",
    "cosmonaut",
    "pediatrician",
    "astronaut",
    "entomologist",
    "cardiologist",
    "doctor",
    "nurse",
    "immunologist",
    "meteorologist",
    "medical researcher",
    "ornithologist",
    "neuroscientist",
    "microbiologist",
    "zoologist",
    "geographer",
    "inventor",
    "geneticist",
    "surgeon",
    "astrophysicist",
    "statistician",
    "sciences",
    "science",
    "mathematics",
    "math",
    "physics",
    "chemistry",
    "biology",
    "epidemiology",
]

business = [
    "executive",
    "businessman",
    "banker",
    "entrepreneur",
    "real estate developer",
    "restaurateur",
    "businesswoman",
    "sports administrator",
    "business",
    "banking",
    "bank",
]
academia_humanities = [
    "scholar",
    "linguist",
    "educator",
    "philosopher",
    "academic",
    "military historian" "historian",
    "educationalist",
    "philologist",
    "librarian",
    "industrialist",
    "professor",
    "musicologist",
    "academia",
    "education",
    "college",
    "university",
    "humanities",
]
law_enf_military_operator = [
    "officer",
    "army",
    "Army",
    "police",
    "admiral",
    "soldier",
    "Air Force",
    "intelligence",
    "major",
    "lieutenant",
    "admiral",
    "fighter pilot",
    "pilot",
    "naval",
    "Navy",
    "aviator",
    "general",
    "CIA",
    "FBI",
    "law enforcement",
    "military",
    "police",
    "Marines",
    "marine",
    "Coast Guard",
]
spiritual = [
    "rabbi",
    "Catholic",
    "priest",
    "Anglican",
    "cardinal",
    "theologian",
    "prelate",
    "Orthodox",
    "Episcopal",
    "bishop",
    "Jesuit",
    "hierarch",
    "Islamic",
    "religious leader",
    "religious",
    "religion",
]
social = ["philanthropist", "socialite", "philanthropy"]
crime = [
    "serial killer",
    "murderer",
    "convicted",
    "terrorist",
    "mobster",
    "criminal",
    "suspect",
    "crime",
    "guilty",
]
event_other = ["Holocaust survivor", "victim", "survivor"]
record = ["supercentenarian", "oldest person", "centarian", "oldest"]
other_species.append("Tree")

<IPython.core.display.Javascript object>

#### Observations:
- We have a good start on `known_for_1` values for which to search.  Some other roles that have been observed previously we have added to the list also.
- Note that roles such as sportswriter and sports broadcaster, though associated with sports, are also included in arts, to align with the underlying nature of the work itself.
- Let us combine them into one dictionary, taking care to put arts last to avoid missing values for "martial artist" and to put spiritual before politics_govt_law so that "leader" in politics_govt_law comes after "religious leader" in relgion.  Likewise "general manager" in sports will come before "general" in law_enf_military_operator and "military historian" in academia_humanities will come before "military" in "law_enf_military_operator".
- We will also include an other_species category here, again putting it last so that trainer and breeder in sports, come before racehorse in other_species.
- Then, we can proceed to extract the category to a new column, `known_for_1`.

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Sets of Values

In [17]:
# Combining separate lists as sets into one dictionary
known_for_dict = {
    "record": set(record),
    "event_other": set(event_other),
    "crime": set(crime),
    "social": set(social),
    "spiritual": set(spiritual),
    "academia_humanities": set(academia_humanities),
    "business": set(business),
    "sciences": set(sciences),
    "sports": set(sports),
    "law_enf_military_operator": set(law_enf_military_operator),
    "politics_govt_law": set(politics_govt_law),
    "arts": set(arts),
    "other_species": set(other_species),
}

<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_1` Column from `info_1`

In [18]:
# Initializing known_for_1 column
df["known_for_1"] = ""

<IPython.core.display.Javascript object>

In [19]:
%%time

# Column to check
column = 'info_1'

# Extract to column
extract_to = 'known_for_1'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 29 values in extract_to column.
CPU times: total: 2.25 s
Wall time: 2.24 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1
99880,13,André Queillé,", 87 French Olympic boxer .",https://en.wikipedia.org/wiki/Andr%C3%A9_Queill%C3%A9,2,2018,December,,Olympic,None,None,None,None,None,None,None,None,None,None,87.0,None,France,None,sports
32981,28,David Turnbull,". 92, American materials scientist.",https://en.wikipedia.org/wiki/David_Turnbull_(materials_scientist),8,2007,April,None,materials,None,None,None,None,None,None,None,None,None,None,92.0,None,United States of America,None,sciences


<IPython.core.display.Javascript object>

#### Observations:
- Once again, the `info_1` column has provided a small sample on which to test our code, which appears to be working.
- We can move on to extracting additional `known_for` values in `info_1` to `known_for_2`.
- Sir Robin Brook is a good example of an individual who would have 3 categories with our approach--business, business, and sports.  So, we will have enough `known_for` columns to extract all values for all entries.  Removing these values has the added benefit of simplifying the next search for `cause_of_death`.

#### Extracting Category to `known_for_2` Column from `info_1`

In [20]:
# Initializing known_for_2 column
df["known_for_2"] = ""

<IPython.core.display.Javascript object>

In [21]:
%%time

# Column to check
column = 'info_1'

# Extract to column
extract_to = 'known_for_2'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_1']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 11 values in extract_to column.
CPU times: total: 3.66 s
Wall time: 3.66 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2
21050,19,Frank Taylor,", 81. English sports journalist.",https://en.wikipedia.org/wiki/Frank_Taylor_(journalist),3,2002,July,None,s,None,None,None,None,None,None,None,None,None,None,81.0,None,United Kingdom of Great Britain and Northern Ireland,None,sports,arts
99323,12,Maryse Morandini,", 85. French Olympic swimmer .",https://en.wikipedia.org/wiki/Maryse_Morandini,2,2018,November,,,None,None,None,None,None,None,None,None,None,None,85.0,None,France,None,sports,sports


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_3` Column from `info_1`

In [22]:
# Initializing known_for_2 column
df["known_for_3"] = ""

<IPython.core.display.Javascript object>

In [23]:
%%time

# Column to check
column = 'info_1'

# Extract to column
extract_to = 'known_for_3'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')

There are 0 values in extract_to column.
CPU times: total: 4.05 s
Wall time: 4.05 s


<IPython.core.display.Javascript object>

In [24]:
# Checking remaining unique values in info_1
df["info_1"].value_counts()

                    18
early                2
Jr                   1
coach                1
player               1
common               1
materials            1
automotive           1
professional         1
aka                  1
gridiron  player     1
Jules Engel          1
s                    1
of                   1
social               1
man                  1
of the Year          1
Name: info_1, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have extracted all of the `known_for` information present in `info_1`.
- It is time to proceed with extracting the same from `info_2`, the column that should contain the bulk of this feature's values.

#### Extracting Category to `known_for_1` Column from `info_2`

In [25]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_1'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 123399 values in extract_to column.
CPU times: total: 3min 32s
Wall time: 3min 32s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3
123415,23,Kulasiri Budawatta,", 71, Sri Lankan dancer and choreographer.",https://en.wikipedia.org/wiki/Kulasiri_Budawatta,24,2021,June,None,None,choreographer,None,None,None,None,None,None,None,None,None,71.0,None,Sri Lanka,None,arts,,
105203,21,Jian Ming,", 58, Chinese poet, writer and literary critic.",https://en.wikipedia.org/wiki/Jian_Ming,2,2019,August,None,None,,writer literary critic,None,None,None,None,None,None,None,None,58.0,None,"China, People's Republic of",None,arts,,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_2` Column from `info_2`

In [26]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_2'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_1']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 34595 values in extract_to column.
CPU times: total: 4min 14s
Wall time: 4min 14s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3
18334,4,V. Appapillai,", 87, Sri Lankan physicist and academic.",https://en.wikipedia.org/wiki/V._Appapillai,5,2001,July,None,None,,None,None,None,None,None,None,None,None,None,87.0,None,Sri Lanka,None,academia_humanities,sciences,
94469,28,Harvey Schmidt,", 88, American musical theatre producer and writer .",https://en.wikipedia.org/wiki/Harvey_Schmidt,7,2018,February,",",None,musical theatre,None,None,None,None,None,None,None,None,None,88.0,None,United States of America,None,arts,arts,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_3` Column from `info_2`

In [27]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_3'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 3311 values in extract_to column.
CPU times: total: 1min 37s
Wall time: 1min 37s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3
26175,7,Herman Postma,", 71, American scientist and educational leader .",https://en.wikipedia.org/wiki/Herman_Postma,15,2004,November,director of Oak Ridge National Laboratory,None,al,None,None,None,None,None,None,None,None,None,71.0,None,United States of America,None,academia_humanities,sciences,politics_govt_law
45940,28,J. J. Maura,", 61, American television announcer and voiceover artist , cancer.",https://en.wikipedia.org/wiki/J._J._Maura,2,2010,July,"WCAU, QVC",None,announcer over,cancer,None,None,None,None,None,None,None,None,61.0,None,United States of America,None,arts,arts,arts


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_4` Column from `info_2`

In [28]:
# Initializing known_for_4 column
df["known_for_4"] = ""

<IPython.core.display.Javascript object>

In [29]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_4'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_3']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 160 values in extract_to column.
CPU times: total: 20.7 s
Wall time: 20.8 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4
73109,13,Tony Ciprian,", 82, New Zealand television sports presenter and producer.",https://en.wikipedia.org/wiki/Tony_Ciprian,6,2015,January,None,None,s,None,None,None,None,None,None,None,None,None,82.0,None,New Zealand,None,sports,arts,arts,arts
72251,2,Carlos Mamery,", 54, Puerto Rican music producer and television personality , heart attack.",https://en.wikipedia.org/wiki/Carlos_Mamery,3,2014,December,,None,,heart attack,None,None,None,None,None,None,None,None,54.0,None,Puerto Rico,None,arts,arts,arts,arts


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_5` Column from `info_2`

In [30]:
# Initializing known_for_5 column
df["known_for_5"] = ""

<IPython.core.display.Javascript object>

In [31]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_5'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_3']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!='']

There are 4 values in extract_to column.
CPU times: total: 23.7 s
Wall time: 23.7 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5
3721,26,Edgar Williams,", 82, British Army military intelligence officer and historian.",https://en.wikipedia.org/wiki/Edgar_Williams,29,1995,June,None,None,,None,None,None,None,None,None,None,None,None,82.0,None,United Kingdom of Great Britain and Northern Ireland,None,law_enf_military_operator,law_enf_military_operator,law_enf_military_operator,law_enf_military_operator,arts
28188,12,Charlie Norman,", 84, Swedish jazz pianist and film music writer.",https://en.wikipedia.org/wiki/Charlie_Norman,14,2005,August,None,None,,None,None,None,None,None,None,None,None,None,84.0,None,Sweden,None,arts,arts,arts,arts,arts
30095,3,Howard Thomas Markey,", 85, American federal judge and U.S. Air Force major general, first chief judge of the United States Court of Appeals for the Federal Circuit.",https://en.wikipedia.org/wiki/Howard_Thomas_Markey,3,2006,May,None,None,,first chief judge of the Court of Appeals for the Federal Circuit,None,None,None,None,None,None,None,None,85.0,None,United States of America,None,law_enf_military_operator,law_enf_military_operator,law_enf_military_operator,politics_govt_law,politics_govt_law
57402,11,Beano Cook,", 81, American college football historian and television sports analyst .",https://en.wikipedia.org/wiki/Beano_Cook,9,2012,October,ESPN,None,s analyst,None,None,None,None,None,None,None,None,None,81.0,None,United States of America,None,academia_humanities,sports,sports,arts,arts


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_6` Column from `info_2`

In [32]:
# Initializing known_for_6 column
df["known_for_6"] = ""

<IPython.core.display.Javascript object>

In [33]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_6'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_3']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')

There are 0 values in extract_to column.
CPU times: total: 19.4 s
Wall time: 19.4 s


<IPython.core.display.Javascript object>

In [34]:
# Checking the number of missing values in known_for_1
print(
    f'There are {len(df[df["known_for_1"] == ""])} missing values in known_for_1 column.'
)

There are 9253 missing values in known_for_1 column.


<IPython.core.display.Javascript object>

#### Observations:
- We have exhuasted our search of `info_2` using the current version of `known_for_dict` and have under 10,000 remaining missing values in `known_for_1`.
- Let us examine some of the remaining unique values in `info_2` and ammend our lists and dictionary.

#### Checking Remaining `info_2` Values

In [35]:
# Obtaining values for column and their counts
col_values = df[df["known_for_1"] == ""]["info_2"].value_counts()

# Creating a list for values that occur more than set number of time
roles_list = [index for index in col_values.index if col_values[index] > 2]

# Checking length of list
print(f"We will examine the top {len(roles_list)} unique values in info_2.")

We will examine the top 559 unique values in info_2.


<IPython.core.display.Javascript object>

In [36]:
# # Using pop to check list items and add to associated dictionary below
# roles_list.pop()

<IPython.core.display.Javascript object>

#### Updating Category Lists for `known_for_dict`

In [60]:
# Appending category lists
sciences = sciences + [
    "volcanologist",
    "gerontologist",
    "pollster",
    "genealogist",
    "software developer",
    "video game developer",
    "anaesthetist",
    "geomorphologist",
    "carcinologist",
    "weatherman",
    "aerodynamicist",
    "limnologist",
    "control theorist",
    "plant pathologist",
    "pathologist",
    "medical practitioner",
    "optometrist",
    "neuroendocrinologist",
    "endocrinologist",
    "anesthesiologist",
    "obstetrician",
    "zookeeper",
    "game developer",
    "forester",
    "embryologist",
    "urologist",
    "arachnologist",
    "lichenologist",
    "anatomist",
    "mineralogist",
    "gastroenterologist",
    "sexologist",
    "bacteriologist",
    "gynecologist",
    "horticulturalist",
    "seismologist",
    "parasitologist",
    "neurophysiologist",
    "primatologist",
    "hydrologist",
    "indologist",
    "ethologist",
    "herbalist",
    "econometrician",
    "cryptographer",
    "toxicologist",
    "haematologist",
    "hematologist",
    "plant ecologist",
    "ecologist",
    "ufologist",
    "crystallographer",
    "gynaecologist",
    "climatologist",
    "glaciologist",
    "demographer",
    "dentist",
    "archeologist",
    "ichthyologist",
    "nephrologist",
    "dermatologist",
    "veterinarian",
    "physiologist",
    "horticulturist",
    "cancer researcher",
    "urban planner",
    "nutritionist",
    "pharmac",
    "oncologist",
    "metallurgist",
    "herpetologist",
    "ophthalmologist",
    "palaeontologist",
    "oceanographer",
    "agronomist",
    "paediatrician",
    "mycologist",
    "naturalist",
    "criminologist",
    "epidemiologist",
    "psychotherapist",
    "neurologist",
    "paleontologist",
    "virologist",
    "psychoanalyst",
]
politics_govt_law = politics_govt_law + [
    "justice",
    "anarchist",
    "secretary",
    "partisan",
    "resistance",
    "Resistance",
    "foreign policy",
    "chieftain",
    "communist",
    "Trotskyist",
    "herald",
    "human rights",
    "campaigner",
    "prince",
    "insurgent",
    "detainee",
    "Resistance",
    "revolutionary",
    "elder",
    "Governor",
    "General",
    "Vice",
    "Admiral",
    "peer",
    "landowner",
    "union",
    "sultan",
    "Sultan",
    "Senator",
    "Representative",
    "loyalist",
    "Supreme Court",
    "Justice",
    "Chief Justice",
    "Conservative",
    "conservative",
    "Liberal",
    "liberal",
    "MP",
    "parliamentarian",
    "pariliament",
    "Parliament",
    "colonial",
    "mayor",
    "Mayor",
    "ruler",
    "republican",
    "Republican",
    "Democrat",
    "democrat",
    "bureaucrat",
    "conspiracy theorist",
    "jihadist",
    "whistleblower",
    "prime minister",
    "countess",
    "District",
    "Judge",
    "foreign minister",
    "Foreign Minister",
    "Islamist",
    "peeress",
    "legislator",
    "first lady",
    "First Lady",
    "courtier",
    "senior",
    "monarch",
    "statesman",
    "lobbyist",
    "solicitor",
    "senator",
    "representative",
    "nationalist",
    "protester",
    "noble",
    "prosecutor",
    "magistrate",
    "public official",
    "feminist",
    "dissident",
    "candidate",
    "congress",
    "administrator",
    'president',
    'Politician'
]
law_enf_military_operator = law_enf_military_operator + [
    "veteran",
    "forester",
    "Navajo code talker",
    "security",
    "fighter",
    "paramilitary",
    "guerrilla",
    "fighter ace",
    "flying ace",
    "firefighter",
    "Medal of Honor",
    "secret agent",
    "codebreaker",
    "Special Operations",
    "warlord",
    "Victoria Cross",
    "mercenary",
    "World War II",
    "colonel",
    "Marine",
    "Secret Service",
    "commander",
    "Air Chief",
    "Marshal",
    "marshal",
    "aviation",
    "airman",
    "spy",
]
sports = sports + [
    "sport",
    "jumper",
    "athletic",
    "shot putter",
    "Olympian",
    "fencing",
    "bandy",
    "Banty",
    "rodeo",
    "rowing",
    "lacrosse",
    "yoga",
    "futsal",
    "heavyweight",
    "Heavyweight",
    "balloonist",
    "racewalker",
    "hurling",
    "biker",
    "scuba",
    "master of the horse",
    "shogi",
    "Football",
    "softball",
    "free diver",
    "greyhound trainer",
    "goalkeeper",
    "mountain",
    "boxing",
    "hunter",
    "angler",
    "aikidoka",
    "aikido",
    "cave diver",
    "alpinist",
    "powerlifter",
    "karate",
    "rowing",
    "coxswain",
    "skater",
    "skating",
    "Go player",
    "orienteer",
    "orienteer",
    "ten pin",
    "karateka",
    "wrestling",
    "announcer",
    "golf",
    "netball",
    "poker",
    "slalom",
    "canoe",
    "pool player",
    "NFL",
    "CFL",
    "CFL",
    "bowl",
    "pole vault",
    "strongman",
    "yachtsman",
    "snowboard",
    "skateboard",
    "archer",
    "climber",
    "swim",
    "squash",
    "climber",
    "shot put",
    "luger",
    "walker",
    "adventurer",
    "diver",
    "surfer",
    "explorer",
    "bullfighter",
    "sprint",
    "pitcher",
]

academia_humanities = academia_humanities + [
    "Esperantist",
    "phonetician",
    "vexillologist",
    "Byzantinist",
    "logician",
    "Turkologist",
    "bioethicist",
    "Mayanist",
    "Hellenist",
    "crossword compiler",
    "cruciverbalist",
    "Hispanist",
    "Arabist",
    "semiotician",
    "Assyriologist",
    "literary theorist",
    "schoolmaster",
    "schoolteacher",
    "intellectual",
    "organizational theorist",
    "information theorist",
    "orientalist",
    "medievalist",
    "classicist",
    "archivist",
    "museum",
    "numismatist",
    "ethnologist",
    "lexicographer",
    "folklorist",
    "philatelist",
    "sinologist",
    "teacher",
    "Egyptologist",
    "Japanologist",
    "Iranologist",
    "Indologist",
]
business = business + [
    "retailer",
    "grocer",
    "auctioneer",
    "baker",
    "car dealer",
    "clothier",
    "food manufacturer",
    "manufacturer",
    "real estate",
    "shipowner",
    "company director",
    "distiller",
    "financial",
    "financial",
    "finance",
    "media owner",
    "printer",
    "management consultant",
    "investment manager",
    "vintner",
    "brewer",
    "jeweller",
    "shipping magnate",
    "nightclub owner",
    "bookseller",
    "billionaire",
    "stockbroker",
    "farmer",
    "hotel",
    "accountant",
    "property developer",
    "investor",
    "financier",
    "winemaker",
]
crime = crime + [
    "murder suspect",
    "suspect",
    "concentration camp guard",
    "drug dealer",
    "drug lord",
    "convict",
    "drug trafficker",
    "spree killer",
    "gangster",
    "snooker",
]
spiritual = spiritual + [
    "Presbyterian",
    "spiritual",
    "Zen",
    "Buddhist",
    "monk",
    "ayatollah",
    "Ayatollah",
    "psychic",
    "yogi",
    "Marja",
    "Trappist",
    "Christian",
    "missionary",
    "Benedictine",
    "nun",
    "faith",
    "healer",
    "Methodist",
    "archdeacon",
    "Baptist",
    "cleric",
    "televangelist",
    "clergy",
    "astrolog",
    "evangelist",
    "minister",
    "pastor",
    'lama'
]
arts = arts + [
    "milliner",
    "memoirist",
    "columnist",
    "bluegrass",
    "fiddler",
    "perfumer",
    "performer",
    "acting",
    "organ builder",
    "art patron",
    "TV",
    "reporter",
    "Pulitzer Prize",
    "script",
    "santoor",
    "mandolin",
    "oenologist",
    "radio",
    "host",
    "horn player",
    "cameraman",
    "tuba",
    "surfboard shaper",
    "impresario",
    "weaver",
    "oud player",
    "blues",
    "reporter",
    "animal trainer",
    "harmonica",
    "guitar",
    "movie",
    "woodworker",
    "R&B",
    "antique",
    "craftsman",
    "double bass",
    "keyboard",
    "drag queen",
    "trumpet",
    "hairstylist",
    "etiquette",
    "accordion",
    "radio",
    "mural",
    "Calypso",
    "calypso",
    "bassoon",
    "animation",
    "correspondent",
    "taekwondo",
    "potter",
    "studio",
    "illusionist",
    "magici",
    "circus",
    "documentar",
    "YouTube",
    "satirist",
    "beauty pageant",
    "baritone",
    "impressionist",
    "performer",
    "stunt",
    "hairdresser",
    "theatre",
    "announcer",
    "flutist",
    "flute",
    "clown",
    "harp",
    "bass player",
    "blog",
    "vlog",
    "show",
    "ventriloquist",
    "typographer",
    "calligrapher",
    "band manager",
    "tabla",
    "storyteller",
    "arranger",
    "news",
    "curator",
    "violist",
    "printmaker",
    "oboist",
    "sound",
    "beauty queen",
    "literary agent",
    "contralto",
    "ceramicist",
    "vocal",
    "ceramist",
    "banjo",
    "publicist",
    "flautist",
    "harpsichord",
    "decorator",
    "talent",
    "accordionist",
    "casting",
    "stage director",
    "theater",
    "humorist",
    "essayist",
    "biographer",
    "art collector",
    "puppeteer",
    "art dealer",
    "drama",
    "art director",
    "entertainer",
    "percussion",
    "clarinet",
    "director",
    "stage",
    "bandoneon",
    "choir",
    "Choir",
]
social = social + [
    "heir",
    "volunteer",
    "public figure",
    "humanitarian",
    "social worker",
]
event_other = event_other + ["homeless", "student", "teenager", "fan of", "worker", 'child','boy', 'girl', 'employee']
record = record + [
    "longevity claimant",
    "record holder",
    "heaviest",
    "tallest",
    "shortest",
    "oldest",
    "youngest",
    "last",
    "first",
    "centenarian",
]
other_species = other_species + [
    "elephant",
    "Great Dane",
    "greyhound",
    "thoroughbred",
]

<IPython.core.display.Javascript object>

#### Updating `known_for_dict` Dictionary of Category Keys and Specific Role Sets of Values

In [38]:
# Combining separate lists as sets into one dictionary
known_for_dict = {
    "record": set(record),
    "event_other": set(event_other),
    "crime": set(crime),
    "social": set(social),
    "academia_humanities": set(academia_humanities),
    "business": set(business),
    "sciences": set(sciences),
    "sports": set(sports),
    "law_enf_military_operator": set(law_enf_military_operator),
    "politics_govt_law": set(politics_govt_law),
    "arts": set(arts),
    "spiritual": set(spiritual),
    "other_species": set(other_species),
}

<IPython.core.display.Javascript object>

#### Observations:
- Now we will repeat extracting `known_for` values from `info_2` with the updated dictionary.

#### Extracting Category to `known_for_1` Column from `info_2` with Updated `known_for_dict`

In [39]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_1'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 130270 values in extract_to column.
CPU times: total: 52.3 s
Wall time: 52.4 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5,known_for_6
20987,9,Madron Seligman,", 83, British politician.",https://en.wikipedia.org/wiki/Madron_Seligman,4,2002,July,None,None,,None,None,None,None,None,None,None,None,None,83.0,None,United Kingdom of Great Britain and Northern Ireland,None,politics_govt_law,,,,,
11035,31,Henry George Glyde,", 91, Canadian painter.",https://en.wikipedia.org/wiki/Henry_George_Glyde,4,1998,March,None,None,,None,None,None,None,None,None,None,None,None,91.0,None,Canada,None,arts,,,,,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_2` Column from `info_2` with Updated `known_for_dict`

In [41]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_2'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_1']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 43467 values in extract_to column.
CPU times: total: 13min
Wall time: 13min


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5,known_for_6
2987,22,Robert Beauchamp,", 71-72, American figurative painter and arts educator, prostate cancer.",https://en.wikipedia.org/wiki/Robert_Beauchamp,4,1995,March,None,None,figurative,prostate cancer,None,None,None,None,None,None,None,None,71.5,None,United States of America,None,academia_humanities,arts,arts,,,
10104,4,Morton Bard,", 73, American psychologist and author of The Crime Victim's Book.",https://en.wikipedia.org/wiki/Morton_Bard,7,1997,December,None,None,of The Crime Victim Book,None,None,None,None,None,None,None,None,None,73.0,None,United States of America,None,sciences,arts,,,,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_3` Column from `info_2` with Updated `known_for_dict`

In [42]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_3'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 6327 values in extract_to column.
CPU times: total: 5min 18s
Wall time: 5min 18s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5,known_for_6
79269,26,William O'Callaghan,", 94, Irish Army lieutenant general, Force Commander .",https://en.wikipedia.org/wiki/William_O%27Callaghan_(Irish_Army_officer),7,2015,December,United Nations Interim Force in,None,,Force Commander,None,None,None,None,None,None,None,None,94.0,None,Ireland,None,law_enf_military_operator,law_enf_military_operator,law_enf_military_operator,,,
78134,19,Patricia Kern,", 88, British mezzo-soprano and voice teacher.",https://en.wikipedia.org/wiki/Patricia_Kern,3,2015,October,None,None,mezzo,None,None,None,None,None,None,None,None,None,88.0,None,United Kingdom of Great Britain and Northern Ireland,None,arts,arts,academia_humanities,,,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_4` Column from `info_2` with Updated `known_for_dict`

In [43]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_4'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 550 values in extract_to column.
CPU times: total: 6min 8s
Wall time: 6min 8s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5,known_for_6
117155,11,Irena Veisaitė,", 92, Lithuanian theatre scholar and human rights activist, COVID-19.",https://en.wikipedia.org/wiki/Irena_Veisait%C4%97,3,2020,December,None,None,,COVID,None,None,None,None,None,None,None,None,92.0,None,Lithuania,None,academia_humanities,politics_govt_law,politics_govt_law,arts,,
30185,17,Eric Forth,", 61, British Conservative Member of Parliament and former government minister, bone cancer.",https://en.wikipedia.org/wiki/Eric_Forth,12,2006,May,None,None,Member of former,bone cancer,None,None,None,None,None,None,None,None,61.0,None,United Kingdom of Great Britain and Northern Ireland,None,politics_govt_law,politics_govt_law,politics_govt_law,spiritual,,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_5` Column from `info_2` with Updated `known_for_dict`

In [44]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_5'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 29 values in extract_to column.
CPU times: total: 5min 9s
Wall time: 5min 9s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5,known_for_6
30443,19,Melvin Watson,", 98, American Baptist minister who trained Martin Luther King Jr. and other civil rights leaders, complications from surgery.",https://en.wikipedia.org/wiki/Melvin_Watson,4,2006,June,None,None,who trained Martin Luther Jr other s,complications from surgery,None,None,None,None,None,None,None,None,98.0,None,United States of America,None,politics_govt_law,politics_govt_law,politics_govt_law,spiritual,spiritual,
24749,26,Jack Sperling,", 81, American jazz drummer who performed in big bands and as a studio musician for movies and television.",https://en.wikipedia.org/wiki/Jack_Sperling,2,2004,February,None,None,who performed in big bands as a studio for s,None,None,None,None,None,None,None,None,None,81.0,None,United States of America,None,arts,arts,arts,arts,arts,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_6` Column from `info_2` with Updated `known_for_dict`

In [46]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_6'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 2 values in extract_to column.
CPU times: total: 4min 52s
Wall time: 4min 52s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5,known_for_6
6088,18,Chet Forte,", 60, American television director and sports radio talk show host, heart attack.",https://en.wikipedia.org/wiki/Chet_Forte,12,1996,May,None,None,s talk,heart attack,None,None,None,None,None,None,None,None,60.0,None,United States of America,None,sports,arts,arts,arts,arts,arts
24749,26,Jack Sperling,", 81, American jazz drummer who performed in big bands and as a studio musician for movies and television.",https://en.wikipedia.org/wiki/Jack_Sperling,2,2004,February,None,None,who performed in big bands as a for s,None,None,None,None,None,None,None,None,None,81.0,None,United States of America,None,arts,arts,arts,arts,arts,arts


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_7` Column from `info_2` with Updated `known_for_dict`

In [48]:
# Initializing known_for_7 column
df["known_for_7"] = ""

<IPython.core.display.Javascript object>

In [53]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_7'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!='']

There are 0 values in extract_to column.
CPU times: total: 4min 57s
Wall time: 4min 57s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5,known_for_6,known_for_7


<IPython.core.display.Javascript object>

In [55]:
len(df[df["known_for_1"] == ""])

2382

<IPython.core.display.Javascript object>

In [57]:
df[df["known_for_1"] == ""]["info_2"].value_counts().head(100)

                             35
agent                         7
president                     7
harpist                       7
bass player                   7
child                         6
bassoonist                    5
woman                         5
lama                          5
official                      4
Politician                    3
sarod player                  3
calypsonian                   3
imam                          3
rejoneador                    2
pornographer                  2
investment advisor            2
hammered dulcimer player      2
theorist                      2
pedagogue                     2
Waffen SS member              2
co creator of                 2
co founder of                 2
fugitive                      2
arms dealer                   2
ski jumping coach             2
bucking bull                  2
Chief of the Air Staff        2
bibliographer                 2
serial burglar                2
land developer                2
gendarme

<IPython.core.display.Javascript object>

In [65]:
df[(df["known_for_1"] == "") & (df["info_2"] == "employee")]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3,known_for_4,known_for_5,known_for_6,known_for_7
28727,27,Jean-Claude Irvoas,", 56, French employee, murder.",https://en.wikipedia.org/wiki/2005_French_riots#Murders_of_Jean-Claude_Irvoas_and_Jean-Jacques_Le_Chenadec,62,2005,October,None,None,employee,murder,None,None,None,None,None,None,None,None,56.0,None,France,None,,,,,,,
51335,6,Shareef Allman,", 47, American employee, suicide by gunshot.",https://en.wikipedia.org/wiki/Permanente_Quarry#2011_shooting,25,2011,October,None,None,employee,suicide by gunshot,None,None,None,None,None,None,None,None,47.0,None,United States of America,None,,,,,,,


<IPython.core.display.Javascript object>

In [54]:
print("dunzo!")
chime.success()

dunzo!


<IPython.core.display.Javascript object>

In [ ]:
test = "Japanologist"

In [ ]:
test.startswith("Japan" + " ")

In [ ]:
test.endswith("logist")

In [ ]:
"Japan" in test

In [63]:
"CIA" in law_enf_military_operator

True

<IPython.core.display.Javascript object>

In [ ]:
[item ]